In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [3]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation, apply_checking_circuit
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.5.1', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

simulation_backend = "qasm_simulator"

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibm_auckland')
device

<IBMQBackend('ibm_auckland') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

100000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_BeH2_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30478/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐   ┌─────────────────┐»
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┤                           │         »
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └────┬───┬─────┘ ├───────────────┴┐┌────────────┐                      »
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
       ┌────┴───┴─────┐ ├────────────────┤└────────────┘                      »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├────────────────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘                                    »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └─────────────────┘  └────────────┘                                      »
q_6: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«      ┌──────────────┐                                                     »
«q_0: ─┤ Rz(-0.23807) ├─────────────────────────────────────────────────────»
«     ┌┴──────────────┤┌──────────────┐                                     »
«q_1: ┤ Ry(0.0055856) ├┤ Rz(-0.21686) ├─────────────────────────────────────»
«     └───────────────┘├─────────────┬┘  ┌────────────┐                     »
«q_2: ────────■────────┤ Ry(0.02067) ├───┤ Rz(1.2696) ├─────────────────────»
«             │        └─────────────┘ ┌─┴────────────┴─┐  ┌──────────────┐ »
«q_3: ────────■───────────────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├─»
«                             │        └────────────────┘  ├──────────────┤ »
«q_4: ────────────────────────■────────────────■───────────┤ Ry(0.025251) ├─»
«                                              │         ┌─┴──────────────┴┐»
«q_5: ─────────────────────────────────────────■─────────┤ Ry(-1.0324e-05) ├»
«                                                        └─────────────────┘»
«q_6: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«                      
«q_2: ─────────────────
«                      
«q_3: ─────────────────
«       ┌───────────┐  
«q_4: ──┤ Rz(1.011) ├──
«     ┌─┴───────────┴─┐
«q_5: ┤ Rz(-0.046623) ├
«     └───────────────┘
«q_6: ─────────────────
«

In [16]:
#checking circuit

In [17]:
phase_y_front = [2.113295958493177e-07,0.0054976714345856634, 0.020630778702533964, 3.40008274438278e-05, 0.02524649597343495, -4.81317308598186e-05]
phase_z_front = [1.3165147216569606, -0.07815775859934079, 0.006091440853078919, 1.528289688855932, -0.009893684371252858, 1.3655876587592497]
phase_y_end = [-7.269945363221252e-06, 0.0055856209568994186, 0.020669506733621706, 2.6996665334365784e-05, 0.02525084679082589, -1.0323843699871446e-05]
phase_z_end = [-0.23806644958543488, -0.21685946937956296, 1.2696203088392648, 0.02265810653600898, 1.010960593326885, -0.04662312860318999]
x_front = [True, False, False, True, False, False]

In [18]:
qubit_id = 2
circuit = QuantumCircuit(7)
apply_checking_circuit(circuit, [qubit_id], [6], side = 'front', phase_y = phase_y_front[qubit_id], phase_z = phase_z_front[qubit_id], x = x_front[qubit_id])
circuit += orign_circuit
apply_checking_circuit(circuit, [qubit_id], [6], side = 'end', phase_y = phase_y_end[qubit_id], phase_z = phase_z_end[qubit_id])

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30478/2877670120.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += orign_circuit


In [19]:
circuit.draw()

┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────┤ H ├───────────────────────────────■─────────────────────────»
            └───┘                                                         »
«     ┌─────────────────┐┌──────────────┐                                     »
«q_0: ┤ Ry(-7.2699e-06) ├┤ Rz(-0.23807) ├─────────────────────────────────────»
«     └─────────────────┘└──────────────┘                    ┌───────────────┐»
«q_1: ─────────────────────────────────────────────────────■─┤ Ry(0.0055856) ├»
«      ┌───────────────┐ ┌──────────────┐┌───────────────┐ │ └───────────────┘»
«q_2: ─┤ Ry(-0.020631) ├─┤ Ry(0.020631) ├┤ Rz(0.0060914) ├─■─────────■────────»
«      └───────────────┘ └──────────────┘└───────────────┘           │        »
«q_3: ───────────────────────────────────────────────────────────────■────────»
«                                                                             »
«q_4: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_5: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐                                                         »
«q_1: ┤ Rz(-0.21686) ├─────────────────────────────────────────────────────────»
«     ├─────────────┬┘  ┌────────────┐    ┌─────────────┐   ┌──────────────┐   »
«q_2: ┤ Ry(0.02067) ├───┤ Rz(1.2696) ├────┤ Rz(-1.2696) ├───┤ Ry(-0.02067) ├─■─»
«     └─────────────┘ ┌─┴────────────┴─┐  ├─────────────┴┐  └──────────────┘ │ »
«q_3: ───────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├───────────────────┼─»
«            │        └────────────────┘  ├──────────────┤   ┌───────────┐   │ »
«q_4: ───────■────────────────■───────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┼─»
«                             │         ┌─┴──────────────┴┐┌─┴───────────┴─┐ │ »
«q_5: ────────────────────────■─────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├─┼─»
«                                       └─────────────────┘└───────────────┘ │ »
«q_6: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«                                  
«q_0: ─────────────────────────────
«                                  
«q_1: ─────────────────────────────
«     ┌─────────────┐┌────────────┐
«q_2: ┤ Ry(0.02067) ├┤ Rz(1.2696) ├
«     └─────────────┘└────────────┘
«q_3: ─────────────────────────────
«                                  
«q_4: ─────────────────────────────
«                                  
«q_5: ─────────────────────────────
«          ┌───┐         

In [20]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [21]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 6,num_qargs = 7)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YXYYXY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
                     
YXYZXZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_1: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_2: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_3: ────────────────
      ┌────────────┐ 
q_4: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_5: ────────────────
                     
q_6: ────────────────
                     
ZXZYXY
                     
q_0: ────────────────
      ┌────────────┐ 
q_1: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_2: ────────────────
     ┌──────────────┐
q_3: ┤ U(π/2,0,π/2) ├
     └┬────────────┬┘
q_4: ─┤ U(π/2,0,π) ├─
     ┌┴────────────┴┐
q_5: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_6: ────────────────
           

In [22]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_30478/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [23]:
circuit_list[-2].draw()

┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────┤ H ├───────────────────────────────■─────────────────────────»
            └───┘                                                         »
«     ┌─────────────────┐┌──────────────┐ ┌──────────────┐                    »
«q_0: ┤ Ry(-7.2699e-06) ├┤ Rz(-0.23807) ├─┤ U(π/2,0,π/2) ├────────────────────»
«     └─────────────────┘└──────────────┘ └──────────────┘   ┌───────────────┐»
«q_1: ─────────────────────────────────────────────────────■─┤ Ry(0.0055856) ├»
«      ┌───────────────┐ ┌──────────────┐┌───────────────┐ │ └───────────────┘»
«q_2: ─┤ Ry(-0.020631) ├─┤ Ry(0.020631) ├┤ Rz(0.0060914) ├─■─────────■────────»
«      └───────────────┘ └──────────────┘└───────────────┘           │        »
«q_3: ───────────────────────────────────────────────────────────────■────────»
«                                                                             »
«q_4: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_5: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_6: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«                                                                              »
«q_0: ─────────────────────────────────────────────────────────────────────────»
«     ┌──────────────┐ ┌──────────────┐                                        »
«q_1: ┤ Rz(-0.21686) ├─┤ U(π/2,0,π/2) ├────────────────────────────────────────»
«     ├─────────────┬┘ └┬────────────┬┘   ┌─────────────┐   ┌──────────────┐   »
«q_2: ┤ Ry(0.02067) ├───┤ Rz(1.2696) ├────┤ Rz(-1.2696) ├───┤ Ry(-0.02067) ├─■─»
«     └─────────────┘ ┌─┴────────────┴─┐  ├─────────────┴┐  ├──────────────┤ │ »
«q_3: ───────■────────┤ Ry(2.6997e-05) ├──┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├─┼─»
«            │        └────────────────┘  ├──────────────┤  └┬───────────┬─┘ │ »
«q_4: ───────■────────────────■───────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┼─»
«                             │         ┌─┴──────────────┴┐┌─┴───────────┴─┐ │ »
«q_5: ────────────────────────■─────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├─┼─»
«                                       └─────────────────┘└───────────────┘ │ »
«q_6: ───────────────────────────────────────────────────────────────────────■─»
«                                                                              »
«                                   
«q_0: ──────────────────────────────
«                                   
«q_1: ──────────────────────────────
«     ┌─────────────┐ ┌────────────┐
«q_2: ┤ Ry(0.02067) ├─┤ Rz(1.2696) ├
«     └─────────────┘ └────────────┘
«q_3: ──────────────────────────────
«     ┌──────────────┐              
«q_4: ┤ U(π/2,0,π/2) ├──────────────
«     └──────────────┘              
«q_5: ──────────────────────────────
«          ┌─

In [24]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [25]:
cuts = [(circuit.qubits[2], 5),(circuit.qubits[2], 11)]

In [26]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

...

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

            ┌───┐       ┌────────────────┐┌────────────┐                  »
q_0: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├────────■─────────»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘        │         »
q_1: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├───────────────────────■─────────»
      └┬──────────────┤ ├───────────────┤               ┌────────────────┐»
q_2: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────■───────┤ Rz(-0.0060914) ├»
       └────┬───┬─────┘ ├───────────────┴┐      │       └─┬────────────┬─┘»
q_3: ───────┤ X ├───────┤ Ry(3.4001e-05) ├──────┼─────────┤ Rz(1.5283) ├──»
       ┌────┴───┴─────┐ ├────────────────┤      │         └────────────┘  »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────┼─────────────────────────»
     ┌─┴──────────────┴┐└─┬────────────┬─┘      │                         »
q_5: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├────────┼─────────────────────────»
     └──────┬───┬──────┘  └────────────┘        │                         »
q_6: ───────

In [27]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 2)),), Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 3)),), Qubit(QuantumRegister(7, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(6, 'q'), 5)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),), Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 4)),), Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 1)),), Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}


In [28]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [29]:
total_variants

24

In [30]:
shots = 10000 * total_variants

In [31]:
fragments_list[0][0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
     ┌──────┴───┴──────┐└─┬────────────┬─┘└────────────┘                      »
q_1: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐   ┌─────────────────┐»
q_2: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_3: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┴┐                          │         »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────────────────────────┼─────────»
       ├──────────────┤ ├───────────────┬┘                          │         »
q_5: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └──────────────┘ └───────────────┘                                     »
«                                                       ┌────────────────┐»
«q_0: ──────────────────■───────────────────────■───────┤ Ry(2.6997e-05) ├»
«                       │                       │       └────────────────┘»
«q_1: ──────────────────┼───────────────────────┼─────────────────────────»
«      ┌──────────────┐ │ ┌──────────────┐      │                         »
«q_2: ─┤ Rz(-0.23807) ├─┼─┤ U(π/2,0,π/2) ├──────┼─────────────────────────»
«     ┌┴──────────────┤ │ ├──────────────┤      │         ┌────────────┐  »
«q_3: ┤ Ry(0.0055856) ├─┼─┤ Rz(-0.21686) ├──────┼─────────┤ U(π/2,0,π) ├──»
«     └───────────────┘ │ └──────────────┘      │         └────────────┘  »
«q_4: ──────────────────┼───────────────────────■─────────────────────────»
«                       │ ┌─────────────┐ ┌────────────┐                  »
«q_5: ──────────────────■─┤ Ry(0.02067) ├─┤ Rz(1.2696) ├──────────────────»
«                         └─────────────┘ └────────────┘                  »
«     ┌──────────────┐  ┌──────────────┐                                  
«q_0: ┤ Rz(0.022658) ├──┤ U(π/2,0,π/2) ├──────────────────────────────────
«     └──────────────┘┌─┴──────────────┴┐┌───────────────┐┌──────────────┐
«q_1: ───────■────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«            │        └─────────────────┘└───────────────┘└──────────────┘
«q_2: ───────┼────────────────────────────────────────────────────────────
«            │                                                            
«q_3: ───────┼────────────────────────────────────────────────────────────
«            │          ┌──────────────┐   ┌───────────┐   ┌────────────┐ 
«q_4: ───────■──────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├───┤ U(π/2,0,π) ├─
«                       └──────────────┘   └───────────┘   └────────────┘ 
«q_5: ────────────────────────────────────────────────────────────────────
«

In [32]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
     ┌──────┴───┴──────┐└─┬────────────┬─┘└────────────┘                      »
q_1: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐   ┌─────────────────┐»
q_2: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_3: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┴┐                          │         »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────────────────────────┼─────────»
       ├──────────────┤ ├───────────────┬┘                          │         »
q_5: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └──────────────┘ └───

            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
     ┌──────┴───┴──────┐└─┬────────────┬─┘└────────────┘                      »
q_1: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐   ┌─────────────────┐»
q_2: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_3: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┴┐                          │         »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────────────────────────┼─────────»
       ├──────────────┤ ├───────────────┬┘                          │         »
q_5: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └──────────────┘ └───────────────

     ┌──────────────┐┌───────────────┐   ┌────────────────┐┌───────────────┐»
q_0: ┤ Ry(0.020631) ├┤ Rz(0.0060914) ├─■─┤ Rz(-0.0060914) ├┤ Ry(-0.020631) ├»
     ├─────────────┬┘└┬──────────────┤ │ └────────────────┘└┬─────────────┬┘»
q_1: ┤ Rz(-1.2696) ├──┤ Ry(-0.02067) ├─┼─────────■──────────┤ Ry(0.02067) ├─»
     └────┬───┬────┘  └──────────────┘ │         │          └────┬───┬────┘ »
q_2: ─────┤ H ├────────────────────────■─────────■───────────────┤ H ├──────»
          └───┘                                                  └───┘      »
«                                   
«q_0: ──────────────────────────────
«     ┌────────────┐┌──────────────┐
«q_1: ┤ Rz(1.2696) ├┤ U(π/2,0,π/2) ├
«     └────────────┘└──────────────┘
«q_2: ──────────────────────────────
«                                   

fragment 0:
            ┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
     ┌──────┴───┴─────

In [33]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [34]:
wire_path_map

{Qubit(QuantumRegister(7, 'q'), 0): ((0, Qubit(QuantumRegister(6, 'q'), 2)),),
 Qubit(QuantumRegister(7, 'q'), 1): ((0, Qubit(QuantumRegister(6, 'q'), 3)),),
 Qubit(QuantumRegister(7, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(6, 'q'), 5)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(7, 'q'), 3): ((0, Qubit(QuantumRegister(6, 'q'), 0)),),
 Qubit(QuantumRegister(7, 'q'), 4): ((0, Qubit(QuantumRegister(6, 'q'), 4)),),
 Qubit(QuantumRegister(7, 'q'), 5): ((0, Qubit(QuantumRegister(6, 'q'), 1)),),
 Qubit(QuantumRegister(7, 'q'), 6): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}

In [35]:
num_fragments = len(fragments)
num_fragments

2

In [36]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

34

In [37]:
fragments[0].draw()

┌───┐       ┌────────────────┐┌────────────┐                      »
q_0: ───────┤ X ├───────┤ Ry(3.4001e-05) ├┤ Rz(1.5283) ├──────────────────────»
     ┌──────┴───┴──────┐└─┬────────────┬─┘└────────────┘                      »
q_1: ┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──────────────────────────────────────»
     └──────┬───┬──────┘┌─┴────────────┴─┐┌────────────┐   ┌─────────────────┐»
q_2: ───────┤ X ├───────┤ Ry(2.1133e-07) ├┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├»
      ┌─────┴───┴─────┐ ├───────────────┬┘└────────────┘ │ └─────────────────┘»
q_3: ─┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├────────────────■──────────■─────────»
      └┬──────────────┤ ├───────────────┴┐                          │         »
q_4: ──┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├──────────────────────────┼─────────»
       ├──────────────┤ ├───────────────┬┘                          │         »
q_5: ──┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├───────────────────────────■─────────»
       └──────────────┘ └───────────────┘                                     »
«                                                       ┌────────────────┐»
«q_0: ──────────────────■───────────────────────■───────┤ Ry(2.6997e-05) ├»
«                       │                       │       └────────────────┘»
«q_1: ──────────────────┼───────────────────────┼─────────────────────────»
«      ┌──────────────┐ │                       │                         »
«q_2: ─┤ Rz(-0.23807) ├─┼───────────────────────┼─────────────────────────»
«     ┌┴──────────────┤ │ ┌──────────────┐      │        ┌──────────────┐ »
«q_3: ┤ Ry(0.0055856) ├─┼─┤ Rz(-0.21686) ├──────┼────────┤ U(π/2,0,π/2) ├─»
«     └───────────────┘ │ └──────────────┘      │        └──────────────┘ »
«q_4: ──────────────────┼───────────────────────■─────────────────────────»
«                       │ ┌─────────────┐ ┌────────────┐                  »
«q_5: ──────────────────■─┤ Ry(0.02067) ├─┤ Rz(1.2696) ├──────────────────»
«                         └─────────────┘ └────────────┘                  »
«     ┌──────────────┐                                                    
«q_0: ┤ Rz(0.022658) ├────────────────────────────────────────────────────
«     └──────────────┘┌─────────────────┐┌───────────────┐┌──────────────┐
«q_1: ───────■────────┤ Ry(-1.0324e-05) ├┤ Rz(-0.046623) ├┤ U(π/2,0,π/2) ├
«            │        └─────────────────┘└───────────────┘└──────────────┘
«q_2: ───────┼────────────────────────────────────────────────────────────
«            │                                                            
«q_3: ───────┼────────────────────────────────────────────────────────────
«            │          ┌──────────────┐   ┌───────────┐  ┌──────────────┐
«q_4: ───────■──────────┤ Ry(0.025251) ├───┤ Rz(1.011) ├──┤ U(π/2,0,π/2) ├
«                       └──────────────┘   └───────────┘  └──────────────┘
«q_5: ────────────────────────────────────────────────────────────────────
«

In [38]:
fragments[1].draw()

┌──────────────┐┌───────────────┐   ┌────────────────┐┌───────────────┐»
q_0: ┤ Ry(0.020631) ├┤ Rz(0.0060914) ├─■─┤ Rz(-0.0060914) ├┤ Ry(-0.020631) ├»
     ├─────────────┬┘└┬──────────────┤ │ └────────────────┘└┬─────────────┬┘»
q_1: ┤ Rz(-1.2696) ├──┤ Ry(-0.02067) ├─┼─────────■──────────┤ Ry(0.02067) ├─»
     └────┬───┬────┘  └──────────────┘ │         │          └────┬───┬────┘ »
q_2: ─────┤ H ├────────────────────────■─────────■───────────────┤ H ├──────»
          └───┘                                                  └───┘      »
«                                   
«q_0: ──────────────────────────────
«     ┌────────────┐┌──────────────┐
«q_1: ┤ Rz(1.2696) ├┤ U(π/2,0,π/2) ├
«     └────────────┘└──────────────┘
«q_2: ──────────────────────────────
«

In [39]:
hardware_index = 0

In [40]:
print(shots)

240000


In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 34 basis
len(tomography_circuits)

34

In [43]:
#contains two fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
#[12,13,14,11,8,9]
#[11,9,12,13,8,14]

In [48]:
multi_exp_circuits[hardware_index][-200].draw()

░        ┌───┐       ┌────────────────┐»
  q_0: ────────────────────░────────┤ X ├───────┤ Ry(3.4001e-05) ├»
                           ░ ┌──────┴───┴──────┐└─┬────────────┬─┘»
  q_1: ────────────────────░─┤ Ry(-4.8132e-05) ├──┤ Rz(1.3656) ├──»
                           ░ └──────┬───┬──────┘┌─┴────────────┴─┐»
  q_2: ────────────────────░────────┤ X ├───────┤ Ry(2.1133e-07) ├»
                           ░  ┌─────┴───┴─────┐ ├───────────────┬┘»
  q_3: ────────────────────░──┤ Ry(0.0054977) ├─┤ Rz(-0.078158) ├─»
                           ░  └┬──────────────┤ ├───────────────┴┐»
  q_4: ────────────────────░───┤ Ry(0.025246) ├─┤ Rz(-0.0098937) ├»
       ┌─────────────────┐ ░   ├──────────────┤ ├───────────────┬┘»
  q_5: ┤ U3(-1.9106,π,0) ├─░───┤ Ry(0.020631) ├─┤ Rz(0.0060914) ├─»
       └─────────────────┘ ░   └──────────────┘ └───────────────┘ »
c34_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_3: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_4: ═══════════════════════════════════════════════════════════»
                                                                  »
c34_5: ═══════════════════════════════════════════════════════════»
                                                                  »
«       ┌────────────┐                                          »
«  q_0: ┤ Rz(1.5283) ├────────────────────────────────────────■─»
«       └────────────┘                                        │ »
«  q_1: ──────────────────────────────────────────────────────┼─»
«       ┌────────────┐   ┌─────────────────┐ ┌──────────────┐ │ »
«  q_2: ┤ Rz(1.3165) ├─■─┤ Ry(-7.2699e-06) ├─┤ Rz(-0.23807) ├─┼─»
«       └────────────┘ │ └─────────────────┘┌┴──────────────┤ │ »
«  q_3: ───────────────■──────────■─────────┤ Ry(0.0055856) ├─┼─»
«                                 │         └───────────────┘ │ »
«  q_4: ──────────────────────────┼───────────────────────────┼─»
«                                 │                           │ »
«  q_5: ──────────────────────────■───────────────────────────■─»
«                                                               »
«c34_0: ════════════════════════════════════════════════════════»
«                                                               »
«c34_1: ════════════════════════════════════════════════════════»
«                                                               »
«c34_2: ════════════════════════════════════════════════════════»
«                                                               »
«c34_3: ════════════════════════════════════════════════════════»
«                                                               »
«c34_4: ════════════════════════════════════════════════════════»
«                                                               »
«c34_5: ════════════════════════════════════════════════════════»
«                                                               »
«                                     ┌────────────────┐┌──────────────┐»
«  q_0: ──────────────────────■───────┤ Ry(2.6997e-05) ├┤ Rz(0.022658) ├»
«                             │       └────────────────┘└──────────────┘»
«  q_1: ──────────────────────┼────────────────────────────────■────────»
«                             │                                │        »
«  q_2: ──────────────────────┼────────────────────────────────┼────────»
«       ┌──────────────┐      │         ┌────────────┐         │        »
«  q_3: ┤ Rz(-0.21686) ├──────┼─────────┤ U(π/2,0,π) ├─────────┼────────»
«       └──────────────┘      │         └────────────┘         │        »
«  q_4: ───

In [61]:
len(multi_exp_circuits[0])

408

In [49]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [11,9,12,13,8,14],
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

jobset_id:  23365a316ba6439b94cf41cd9f0cf2dc-1670513110796591
job_id:  8b423cd3-afb6-4784-9cfb-15fc7d2a3f80
Job Status: job has successfully run


In [122]:
# from qiskit.providers.ibmq.managed import IBMQJobManager
# job_manager = IBMQJobManager()

In [123]:
# job_set = job_manager.retrieve_job_set(job_set_id='973c2fdceaf547389a770d19aad463d8-1669966038684177', provider=provider)

In [124]:
# managed_results = job_set.results()

In [125]:
# frag_datas1_copy = frag_datas[1].copy()

In [51]:
frag_datas[0] = [frag_datas[0][0].combine_results()]

In [52]:
# frag_datas[0] = run_circuits(multi_exp_circuits[0], initial_layout = None,
#                              backend = simulation_backend, shots = shots // total_variants,
#                              optimization_level = 3, monitor_jobs = True, noise_model = None)

In [53]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z'))"

In [54]:
num_tomo_circuits

34

In [55]:
test = [[], []]

In [56]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [57]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [5] [5]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'X')) ('S0',) ('X',) [5] [5]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Y')) ('S0',) ('Y',) [5] [5]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [5] [5]
(('S0', 'S0', 'S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z', 'Z', 'X'

In [58]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [59]:
len(organized_data)

34

In [60]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'00000': {(('S0',), ('Zp',)): 285, (('S0',), ('Zm',)): 3, (('S0',), ('Xp',)): 138, (('S0',), ('Xm',)): 144, (('S0',), ('Yp',)): 223, (('S0',), ('Ym',)): 252, (('S1',), ('Zp',)): 81, (('S1',), ('Zm',)): 111, (('S1',), ('Xp',)): 315, (('S1',), ('Xm',)): 187, (('S1',), ('Yp',)): 42, (('S1',), ('Ym',)): 453, (('S2',), ('Zp',)): 107, (('S2',), ('Zm',)): 118, (('S2',), ('Xp',)): 44, (('S2',), ('Xm',)): 442, (('S2',), ('Yp',)): 302, (('S2',), ('Ym',)): 218, (('S3',), ('Zp',)): 82, (('S3',), ('Zm',)): 121, (('S3',), ('Xp',)): 421, (('S3',), ('Xm',)): 99, (('S3',), ('Yp',)): 436, (('S3',), ('Ym',)): 92}, '00001': {(('S0',), ('Zp',)): 266, (('S0',), ('Zm',)): 2, (('S0',), ('Xp',)): 135, (('S0',), ('Xm',)): 133, (('S0',), ('Yp',)): 214, (('S0',), ('Ym',)): 231, (('S1',), ('Zp',)): 92, (('S1',), ('Zm',)): 142, (('S1',), ('Xp',)): 263, (('S1',), ('Xm',)): 174, (('S1',), ('Yp',)): 34, (('S1',), ('Ym',)): 441, (('S2',), ('Zp',)): 96, (('S2',), ('Zm',)): 141, (('S2',), ('Xp',)): 22, (('S2',), ('Xm',)

{'00': {(('S0',), ('Zm',)): 2, (('S0',), ('Zp',)): 9997, (('S0',), ('Xp',)): 4883, (('S0',), ('Xm',)): 5113, (('S0',), ('Ym',)): 4948, (('S0',), ('Yp',)): 5047, (('S1',), ('Zp',)): 3337, (('S1',), ('Xm',)): 1706, (('S1',), ('Xp',)): 1657, (('S1',), ('Yp',)): 1639, (('S1',), ('Ym',)): 1714, (('S2',), ('Zp',)): 3213, (('S2',), ('Xm',)): 1641, (('S2',), ('Xp',)): 1637, (('S2',), ('Ym',)): 1652, (('S2',), ('Yp',)): 1626, (('S3',), ('Zp',)): 3388, (('S3',), ('Xm',)): 1710, (('S3',), ('Xp',)): 1669, (('S3',), ('Ym',)): 1769, (('S3',), ('Yp',)): 1652, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Xp',)): 1, (('S2',), ('Zm',)): 1, (('S2',), ('Xm',)): 2, (('S2',), ('Ym',)): 1, (('S3',), ('Xm',)): 1, (('S3',), ('Xp',)): 1, (('S0',), ('Zm',)): 0, (('S0',), ('Yp',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S

{'00000': {(('S0',), ('Zp',)): 617, (('S0',), ('Zm',)): 6, (('S0',), ('Xp',)): 285, (('S0',), ('Xm',)): 272, (('S0',), ('Yp',)): 311, (('S0',), ('Ym',)): 314, (('S1',), ('Zp',)): 196, (('S1',), ('Zm',)): 415, (('S1',), ('Xp',)): 313, (('S1',), ('Xm',)): 324, (('S1',), ('Yp',)): 50, (('S1',), ('Ym',)): 549, (('S2',), ('Zp',)): 219, (('S2',), ('Zm',)): 396, (('S2',), ('Xp',)): 56, (('S2',), ('Xm',)): 565, (('S2',), ('Yp',)): 408, (('S2',), ('Ym',)): 185, (('S3',), ('Zp',)): 184, (('S3',), ('Zm',)): 434, (('S3',), ('Xp',)): 560, (('S3',), ('Xm',)): 100, (('S3',), ('Yp',)): 469, (('S3',), ('Ym',)): 153}, '00001': {(('S0',), ('Zp',)): 527, (('S0',), ('Zm',)): 5, (('S0',), ('Xp',)): 362, (('S0',), ('Xm',)): 269, (('S0',), ('Yp',)): 299, (('S0',), ('Ym',)): 303, (('S1',), ('Zp',)): 210, (('S1',), ('Zm',)): 372, (('S1',), ('Xp',)): 270, (('S1',), ('Xm',)): 269, (('S1',), ('Yp',)): 28, (('S1',), ('Ym',)): 555, (('S2',), ('Zp',)): 219, (('S2',), ('Zm',)): 373, (('S2',), ('Xp',)): 68, (('S2',), (

{'11': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Zp',)): 6616, (('S1',), ('Xp',)): 3177, (('S1',), ('Xm',)): 3474, (('S1',), ('Yp',)): 3312, (('S1',), ('Ym',)): 3249, (('S2',), ('Zp',)): 6728, (('S2',), ('Xp',)): 3229, (('S2',), ('Xm',)): 3526, (('S2',), ('Ym',)): 3375, (('S2',), ('Yp',)): 3307, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 6626, (('S3',), ('Xm',)): 3352, (('S3',), ('Xp',)): 3202, (('S3',), ('Yp',)): 3275, (('S3',), ('Ym',)): 3319, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Yp',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0}, '01': {(('S0',), ('Zp',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Ym',)): 2, (('S3',), ('Zm',)): 4, (('S3',), ('Xp',)): 1, (('S3',), ('Yp',)): 2, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Yp',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S2',), ('Zp',)): 0, (('S2',), ('Zm',)):

{'10': {(('S0',), ('Zm',)): 1, (('S0',), ('Xp',)): 1, (('S1',), ('Zp',)): 1, (('S2',), ('Xp',)): 2, (('S2',), ('Yp',)): 1, (('S3',), ('Zm',)): 1, (('S3',), ('Xm',)): 2, (('S3',), ('Yp',)): 2, (('S0',), ('Zp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S1',), ('Ym',)): 0, (('S2',), ('Zp',)): 0, (('S2',), ('Zm',)): 0, (('S2',), ('Xm',)): 0, (('S2',), ('Ym',)): 0, (('S3',), ('Zp',)): 0, (('S3',), ('Xp',)): 0, (('S3',), ('Ym',)): 0}, '01': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Xp',)): 1, (('S0',), ('Yp',)): 1, (('S1',), ('Zm',)): 1, (('S1',), ('Ym',)): 2, (('S2',), ('Xm',)): 2, (('S2',), ('Ym',)): 1, (('S3',), ('Zp',)): 1, (('S0',), ('Zm',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S2',), ('Zp',)): 0, (('S2',), ('Zm',)): 0, (('S2',), ('Xp',)): 0, (('S2',), ('Yp',))

...

{'00': {(('S0',), ('Zm',)): 2, (('S0',), ('Zp',)): 9996, (('S0',), ('Xm',)): 5157, (('S0',), ('Xp',)): 4840, (('S0',), ('Ym',)): 4979, (('S0',), ('Yp',)): 5016, (('S1',), ('Zm',)): 1, (('S1',), ('Zp',)): 3314, (('S1',), ('Xm',)): 1776, (('S1',), ('Xp',)): 1586, (('S1',), ('Ym',)): 1686, (('S1',), ('Yp',)): 1741, (('S2',), ('Zp',)): 3258, (('S2',), ('Xm',)): 1637, (('S2',), ('Xp',)): 1596, (('S2',), ('Ym',)): 1596, (('S2',), ('Yp',)): 1616, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 3461, (('S3',), ('Xm',)): 1756, (('S3',), ('Xp',)): 1706, (('S3',), ('Yp',)): 1669, (('S3',), ('Ym',)): 1741, (('S2',), ('Zm',)): 0}, '11': {(('S0',), ('Zp',)): 1, (('S0',), ('Xm',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Zp',)): 6682, (('S1',), ('Xp',)): 3283, (('S1',), ('Xm',)): 3353, (('S1',), ('Ym',)): 3351, (('S1',), ('Yp',)): 3220, (('S2',), ('Zp',)): 6742, (('S2',), ('Xp',)): 3342, (('S2',), ('Xm',)): 3425, (('S2',), ('Yp',)): 3355, (('S2',), ('Ym',)): 3431, (('S3',), ('Zp',)): 6537, (('S3',), ('Xp',)

{'00': {(('S0',), ('Zm',)): 1, (('S0',), ('Zp',)): 9996, (('S0',), ('Xm',)): 5187, (('S0',), ('Xp',)): 4808, (('S0',), ('Yp',)): 5077, (('S0',), ('Ym',)): 4920, (('S1',), ('Zp',)): 3319, (('S1',), ('Xm',)): 1625, (('S1',), ('Xp',)): 1676, (('S1',), ('Ym',)): 1638, (('S1',), ('Yp',)): 1704, (('S2',), ('Zm',)): 3, (('S2',), ('Zp',)): 3195, (('S2',), ('Xm',)): 1603, (('S2',), ('Xp',)): 1540, (('S2',), ('Ym',)): 1614, (('S2',), ('Yp',)): 1613, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 3467, (('S3',), ('Xp',)): 1693, (('S3',), ('Xm',)): 1675, (('S3',), ('Yp',)): 1688, (('S3',), ('Ym',)): 1625, (('S1',), ('Zm',)): 0}, '11': {(('S0',), ('Zp',)): 2, (('S0',), ('Xp',)): 1, (('S0',), ('Ym',)): 1, (('S0',), ('Yp',)): 1, (('S1',), ('Zm',)): 3, (('S1',), ('Zp',)): 6677, (('S1',), ('Xm',)): 3414, (('S1',), ('Xp',)): 3281, (('S1',), ('Yp',)): 3354, (('S1',), ('Ym',)): 3301, (('S2',), ('Zp',)): 6799, (('S2',), ('Xp',)): 3313, (('S2',), ('Xm',)): 3540, (('S2',), ('Ym',)): 3464, (('S2',), ('Yp',)): 330

{'00000': {(('S0',), ('Zp',)): 310, (('S0',), ('Zm',)): 3, (('S0',), ('Xp',)): 170, (('S0',), ('Xm',)): 183, (('S0',), ('Yp',)): 139, (('S0',), ('Ym',)): 173, (('S1',), ('Zp',)): 122, (('S1',), ('Zm',)): 143, (('S1',), ('Xp',)): 156, (('S1',), ('Xm',)): 102, (('S1',), ('Yp',)): 49, (('S1',), ('Ym',)): 447, (('S2',), ('Zp',)): 105, (('S2',), ('Zm',)): 126, (('S2',), ('Xp',)): 22, (('S2',), ('Xm',)): 223, (('S2',), ('Yp',)): 284, (('S2',), ('Ym',)): 182, (('S3',), ('Zp',)): 86, (('S3',), ('Zm',)): 137, (('S3',), ('Xp',)): 216, (('S3',), ('Xm',)): 33, (('S3',), ('Yp',)): 382, (('S3',), ('Ym',)): 91}, '00001': {(('S0',), ('Zp',)): 300, (('S0',), ('Zm',)): 2, (('S0',), ('Xp',)): 132, (('S0',), ('Xm',)): 145, (('S0',), ('Yp',)): 148, (('S0',), ('Ym',)): 137, (('S1',), ('Zp',)): 89, (('S1',), ('Zm',)): 97, (('S1',), ('Xp',)): 103, (('S1',), ('Xm',)): 85, (('S1',), ('Yp',)): 31, (('S1',), ('Ym',)): 415, (('S2',), ('Zp',)): 106, (('S2',), ('Zm',)): 100, (('S2',), ('Xp',)): 26, (('S2',), ('Xm',)

{'00000': {(('S0',), ('Zp',)): 1151, (('S0',), ('Zm',)): 15, (('S0',), ('Xp',)): 575, (('S0',), ('Xm',)): 606, (('S0',), ('Yp',)): 554, (('S0',), ('Ym',)): 564, (('S1',), ('Zp',)): 404, (('S1',), ('Zm',)): 856, (('S1',), ('Xp',)): 533, (('S1',), ('Xm',)): 562, (('S1',), ('Yp',)): 47, (('S1',), ('Ym',)): 1013, (('S2',), ('Zp',)): 413, (('S2',), ('Zm',)): 905, (('S2',), ('Xp',)): 104, (('S2',), ('Xm',)): 1029, (('S2',), ('Yp',)): 750, (('S2',), ('Ym',)): 300, (('S3',), ('Zp',)): 382, (('S3',), ('Zm',)): 880, (('S3',), ('Xp',)): 1010, (('S3',), ('Xm',)): 152, (('S3',), ('Yp',)): 861, (('S3',), ('Ym',)): 354}, '00001': {(('S0',), ('Zp',)): 1169, (('S0',), ('Zm',)): 9, (('S0',), ('Xp',)): 614, (('S0',), ('Xm',)): 592, (('S0',), ('Yp',)): 534, (('S0',), ('Ym',)): 556, (('S1',), ('Zp',)): 376, (('S1',), ('Zm',)): 576, (('S1',), ('Xp',)): 572, (('S1',), ('Xm',)): 538, (('S1',), ('Yp',)): 67, (('S1',), ('Ym',)): 1026, (('S2',), ('Zp',)): 375, (('S2',), ('Zm',)): 574, (('S2',), ('Xp',)): 119, ((

{'00': {(('S0',), ('Zm',)): 1, (('S0',), ('Zp',)): 9995, (('S0',), ('Xp',)): 4825, (('S0',), ('Xm',)): 5172, (('S0',), ('Ym',)): 5023, (('S0',), ('Yp',)): 4972, (('S1',), ('Zm',)): 1, (('S1',), ('Zp',)): 3338, (('S1',), ('Xp',)): 1606, (('S1',), ('Xm',)): 1734, (('S1',), ('Ym',)): 1654, (('S1',), ('Yp',)): 1691, (('S2',), ('Zp',)): 3270, (('S2',), ('Xp',)): 1664, (('S2',), ('Xm',)): 1681, (('S2',), ('Ym',)): 1657, (('S2',), ('Yp',)): 1633, (('S3',), ('Zm',)): 1, (('S3',), ('Zp',)): 3392, (('S3',), ('Xp',)): 1594, (('S3',), ('Xm',)): 1758, (('S3',), ('Ym',)): 1736, (('S3',), ('Yp',)): 1726, (('S2',), ('Zm',)): 0}, '10': {(('S0',), ('Zm',)): 1, (('S0',), ('Xp',)): 1, (('S0',), ('Yp',)): 1, (('S0',), ('Ym',)): 1, (('S1',), ('Zp',)): 2, (('S1',), ('Ym',)): 1, (('S2',), ('Zp',)): 1, (('S2',), ('Zm',)): 2, (('S2',), ('Ym',)): 1, (('S2',), ('Yp',)): 1, (('S3',), ('Zm',)): 1, (('S3',), ('Xm',)): 1, (('S0',), ('Zp',)): 0, (('S0',), ('Xm',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S

In [61]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [63]:
from utils.utils import filter_results

In [64]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [65]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [66]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [67]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [68]:
final_expect_val

(-3.393841417959411+0j)

In [69]:
import json
with open('BeH2_auckland_check2.json', 'w') as f:
    json.dump(norm_filtered_lists, f)